In [1]:
import pandas as pd
import numpy as np
import wrds

In [6]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

Enter your WRDS username [ec2-user]: ly229
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


## Mrkt Cap from CRSP daily data(desampled to monthly)

In [3]:
mktcap_crsp = conn.raw_sql("""
                        SELECT DISTINCT 
                        date AS date, 
                        permno, 
                        permco, 
                        cusip,
                        ABS(prc*shrout)/1000 AS mrkcap_CRSP, 
                        SUM(ABS(prc*shrout)/1000) OVER (PARTITION BY date, permco) AS tot_mrkcap_CRSP
                        
                        FROM crsp.dsf
                        
                        WHERE EXTRACT(MONTH FROM date) IN (5,6) 
                        AND date >= '2000-05-15' 
                        AND date <= '2008-06-30'
                        """)
#mktcap_crsp['date'] = pd.to_datetime(mktcap_crsp['date'])

#'select data AS date', otherwise, it is not selected as a column when only 'select date'
#mktcap_crsp.columns

In [8]:
mktcap_crsp['date'] = pd.to_datetime(mktcap_crsp['date'])

In [10]:
mktcap_crsp_resample = mktcap_crsp
mktcap_crsp_resample.set_index('date', inplace=True)
mktcap_crsp_monthly = mktcap_crsp_resample.groupby(['permno', 'permco','cusip']).resample('ME', include_groups=False).mean().reset_index()

In [22]:
file_path_1 = 'RR_data/mktcap_crsp_monthly.csv' 
mktcap_crsp_monthly.to_csv(file_path_1, index=False)

In [4]:
mktcap_crsp

,date,permno,permco,cusip,mrkcap_crsp,tot_mrkcap_crsp
0,2000-05-15,10001,7953,36720410,19.096000,19.096000
1,2000-05-15,10002,7954,05978R10,82.113187,82.113187
2,2000-05-15,10009,7965,46334710,74.544938,74.544938
3,2000-05-15,10012,7969,23326910,120.937500,120.937500
4,2000-05-15,10016,1728,81002230,336.821625,336.821625
...,...,...,...,...,...,...
221091,2008-06-30,92725,44072,46137V53,2.268000,11332.311480
221092,2008-06-30,92807,7407,03074A10,65.115980,65.115980
221093,2008-06-30,92874,7378,92655510,72.051260,72.051260
221094,2008-06-30,92922,39147,78468R62,395.505000,8983.417150


In [5]:
print(mktcap_crsp.loc[mktcap_crsp['permno'] == 10001])

              date  permno  permco     cusip  mrkcap_crsp  tot_mrkcap_crsp
0       2000-05-15   10001    7953  36720410    19.096000        19.096000
8479    2000-05-16   10001    7953  36720410    19.866000        19.866000
16957   2000-05-17   10001    7953  36720410    19.866000        19.866000
25430   2000-05-18   10001    7953  36720410    19.866000        19.866000
33906   2000-05-19   10001    7953  36720410    19.712000        19.712000
...            ...     ...     ...       ...          ...              ...
185795  2008-06-24   10001    7953  36720410    46.121670        46.121670
192852  2008-06-25   10001    7953  36720410    47.599650        47.599650
199915  2008-06-26   10001    7953  36720410    47.443158        47.443158
206977  2008-06-27   10001    7953  36720410    46.730250        46.730250
214038  2008-06-30   10001    7953  36720410    46.751750        46.751750

[374 rows x 6 columns]


In [64]:
different_rows = mktcap_crsp[mktcap_crsp['mrkcap_crsp'] != mktcap_crsp['tot_mrkcap_crsp']]
print(different_rows)

             date  permno  permco mrkcap_crsp tot_mrkcap_crsp
52     2000-05-15   10195    1221     $105.31       $1,082.38
63     2000-05-15   10239     714     $173.09         $211.92
66     2000-05-15   10253    7922        None            None
75     2000-05-15   10294    2089      $56.24         $248.48
84     2000-05-15   10324    8243   $4,282.60       $4,727.19
...           ...     ...     ...         ...             ...
221087 2008-06-30   92721   37493       $5.20     $246,585.65
221089 2008-06-30   92723   37493       $4.73     $246,585.65
221090 2008-06-30   92724   44072       $2.36      $11,332.31
221091 2008-06-30   92725   44072       $2.27      $11,332.31
221094 2008-06-30   92922   39147     $395.50       $8,983.42

[210385 rows x 5 columns]


In [ ]:
print(mktcap_crsp.loc[mktcap_crsp['permco'] == 1221])

In [63]:
unique_count = mktcap_crsp[['permco', 'date']].drop_duplicates().shape[0]
print(f'Number of unique combinations: {unique_count}')

Number of unique combinations: 2607497


In [5]:
mktcap_crsp['date'] = pd.to_datetime(mktcap_crsp['date'])
mktcap_crsp_test = mktcap_crsp[1:100000]
mktcap_crsp_test['date'].dtype

dtype('<M8[ns]')

In [29]:
mktcap_crsp_test_re = mktcap_crsp_test[['date', 'permco', 'tot_mrkcap_crsp']]
mktcap_crsp_test_re

,date,permco,tot_mrkcap_crsp
1,2000-05-15,7954,82.113187
2,2000-05-15,7965,74.544938
3,2000-05-15,7969,120.937500
4,2000-05-15,1728,336.821625
5,2000-05-15,7971,54.596625
...,...,...,...
99995,2000-05-31,15920,40.677937
99996,2000-05-31,15918,75.127500
99997,2000-05-31,15935,32.754375
99998,2000-05-31,15923,11.424000


In [30]:
mktcap_crsp_test_re.set_index('date', inplace=True)
mktcap_crsp_test_m = mktcap_crsp_test_re.groupby('permco').resample('ME', include_groups=False).mean().reset_index()

In [31]:
mktcap_crsp_test_m

,permco,date,tot_mrkcap_crsp
0,5,2000-05-31,58.366276
1,7,2000-05-31,15073.527349
2,25,2000-05-31,6824.310479
3,29,2000-05-31,35233.011417
4,33,2000-05-31,2048.604672
...,...,...,...
8343,55510,2000-05-31,22.010641
8344,56242,2000-05-31,90.018305
8345,56245,2000-05-31,576.476198
8346,56283,2000-05-31,89.276260


In [32]:
print(mktcap_crsp_test.loc[mktcap_crsp_test['permco'] == 1221])

            date  permno  permco     cusip  mrkcap_crsp  tot_mrkcap_crsp
52    2000-05-15   10195    1221  20334920   105.313500      1082.376000
888   2000-05-15   25348    1221  20334910   977.062500      1082.376000
8531  2000-05-16   10195    1221  20334920   106.488000      1121.519250
9367  2000-05-16   25348    1221  20334910  1015.031250      1121.519250
17009 2000-05-17   10195    1221  20334920   105.183000      1098.698625
17845 2000-05-17   25348    1221  20334910   993.515625      1098.698625
25482 2000-05-18   10195    1221  20334920   105.183000      1054.401750
26317 2000-05-18   25348    1221  20334910   949.218750      1054.401750
33958 2000-05-19   10195    1221  20334920   103.356000      1039.918500
34793 2000-05-19   25348    1221  20334910   936.562500      1039.918500
42438 2000-05-22   10195    1221  20334920   103.356000      1014.606000
43273 2000-05-22   25348    1221  20334910   911.250000      1014.606000
50916 2000-05-23   10195    1221  20334920   103.35

In [35]:
print(mktcap_crsp_test_m.loc[mktcap_crsp_test_m['permco'] == 1221])

     permco       date  tot_mrkcap_crsp
180    1221 2000-05-31      1032.402375


In [6]:
mktcap_crsp_test_resample = mktcap_crsp_test
mktcap_crsp_test_resample.set_index('date', inplace=True)
mktcap_crsp_test_monthly = mktcap_crsp_test_resample.groupby(['permno', 'permco','cusip']).resample('ME', include_groups=False).mean().reset_index()

NameError: name 'mktcap_crsp_test' is not defined

In [81]:
print(mktcap_crsp_test_monthly.loc[mktcap_crsp_test_monthly['permco'] == 1221])

     permno  permco     cusip       date  mrkcap_crsp  tot_mrkcap_crsp
52    10195    1221  20334920 2000-05-31    99.636750      1032.402375
888   25348    1221  20334910 2000-05-31   932.765625      1032.402375


In [80]:
print(mktcap_crsp_test_monthly.loc[mktcap_crsp_test_monthly['permco'] == 7922])

    permno  permco     cusip       date  mrkcap_crsp  tot_mrkcap_crsp
66   10253    7922  26253C10 2000-05-31          NaN              NaN


## Mktcap from Compustat - daily security info

### 1. aggregate data by gvkey -- (gvkey-iids to gvkey)

### 2. calculate prccd_vw -- sum(mktcap_COMP*prccd)/sum(mktcap_COMP)

In [11]:
mktcap_comp = conn.raw_sql("""
                        SELECT DISTINCT 
                        datadate AS date,  
                        gvkey,
                        SUM(prccd*cshoc)/NULLIF(SUM(cshoc), 0) AS prccd,
                        SUM(cshoc)/1000000 AS cshoc,
                        SUM(prccd*cshoc*prccd)/NULLIF(SUM(prccd*cshoc), 0) AS prccd_vw
                        
                        FROM comp.secd
                        
                        WHERE EXTRACT(MONTH FROM datadate) IN (5,6) 
                        AND datadate >= '1999-01-01' 
                        AND curcdd = 'USD'
                        AND tpci in ('0')
                        
                        GROUP BY gvkey, datadate
                        """)

In [126]:
mktcap_comp_test = conn.raw_sql("""
                        SELECT DISTINCT 
                        datadate AS date,  
                        gvkey,
                        SUM(prccd*cshoc)/NULLIF(SUM(cshoc), 0) AS prccd,
                        SUM(cshoc)/1000000 AS cshoc,
                        SUM(prccd*cshoc*prccd)/NULLIF(SUM(prccd*cshoc), 0) AS prccd_vw,
                        COUNT(DISTINCT iid) AS iid_count
                        
                        FROM comp.secd
                        
                        WHERE EXTRACT(MONTH FROM datadate) IN (5,6) 
                        AND datadate >= '2009-01-01' 
                        AND datadate <= '2012-01-01'
                        AND curcdd = 'USD'
                        AND tpci in ('0')
                        
                        GROUP BY gvkey, datadate
                        """)

In [127]:
mktcap_comp_test.head(10)

,date,gvkey,prccd,cshoc,prccd_vw,iid_count
0,2009-05-01,001003,0.020000,2.683,0.020000,1
1,2009-05-01,001004,14.580000,38.697,14.580000,1
2,2009-05-01,001013,7.490000,96.611,7.490000,1
3,2009-05-01,001019,250.000000,0.193,250.000000,1
4,2009-05-01,001021,0.040000,17.929,0.040000,1
5,2009-05-01,001045,5.170000,284.953,5.170000,1
6,2009-05-01,001050,3.600000,14.282,3.600000,1
7,2009-05-01,001072,9.870000,170.460,9.870000,1
8,2009-05-01,001075,27.710000,101.069,27.710000,1
9,2009-05-01,001076,32.923826,53.943,33.032148,2


In [129]:
filtered_df = mktcap_comp_test[mktcap_comp_test['iid_count'] >= 2]
filtered_df

,date,gvkey,prccd,cshoc,prccd_vw,iid_count
9,2009-05-01,001076,32.923826,53.943,33.032148,2
14,2009-05-01,001097,20.476955,1.931,21.010179,2
90,2009-05-01,001627,0.250000,3.700,0.250000,2
124,2009-05-01,001823,10.799975,10.775,10.802059,2
150,2009-05-01,001982,19.057559,14.754,19.062843,2
...,...,...,...,...,...,...
316225,2011-06-30,176065,0.284066,13.113,0.284294,2
316313,2011-06-30,176574,1.464438,21.191,1.987495,2
316965,2011-06-30,180821,52.310600,14.398,52.465781,2
317216,2011-06-30,183565,2.296474,7.232,2.303384,2


In [132]:
mktcap_comp.head(10)

,date,gvkey,prccd,cshoc,prccd_vw
0,2016-05-09,020502,13.2400,43.281,13.2400
1,1999-05-03,063286,0.0300,3.150,0.0300
2,2007-06-08,023002,30.2100,44.407,30.2100
3,2005-05-04,064766,14.2400,99.503,14.2400
4,2016-06-02,026123,10.9900,84.520,10.9900
5,2005-05-12,002236,2.4750,0.293,2.4750
6,2009-06-04,064284,0.0075,62.345,0.0075
7,2009-06-11,025711,2.3500,7.414,2.3500
8,2004-06-15,003367,0.2100,13.588,0.2100
9,2003-06-02,001078,43.9300,1560.967,43.9300


In [34]:
unique_date = mktcap_comp['date'].unique()
unique_date

<DatetimeArray>
['1999-05-03 00:00:00', '1999-05-04 00:00:00', '1999-05-05 00:00:00',
 '1999-05-06 00:00:00', '1999-05-07 00:00:00', '1999-05-10 00:00:00',
 '1999-05-11 00:00:00', '1999-05-12 00:00:00', '1999-05-13 00:00:00',
 '1999-05-14 00:00:00',
 ...
 '2012-06-18 00:00:00', '2012-06-19 00:00:00', '2012-06-20 00:00:00',
 '2012-06-21 00:00:00', '2012-06-22 00:00:00', '2012-06-25 00:00:00',
 '2012-06-26 00:00:00', '2012-06-27 00:00:00', '2012-06-28 00:00:00',
 '2012-06-29 00:00:00']
Length: 596, dtype: datetime64[ns]

## resampleing to monthly frequency

In [13]:
mktcap_comp['date'] = pd.to_datetime(mktcap_comp['date'])

In [15]:
mktcap_comp_resample = mktcap_comp
mktcap_comp_resample['date'] = pd.to_datetime(mktcap_comp_resample['date'])
mktcap_comp_resample.set_index('date', inplace=True)
mktcap_comp_re_monthly = mktcap_comp_resample.groupby('gvkey').resample('ME', include_groups=False).mean().reset_index().dropna()

In [135]:
mktcap_comp_re_monthly

,gvkey,date,prccd,cshoc,prccd_vw
0,001003,2002-06-30,0.009000,2.683000,0.009000
11,001003,2003-05-31,0.040000,2.683000,0.040000
12,001003,2003-06-30,0.040000,2.683000,0.040000
23,001003,2004-05-31,0.040000,2.683000,0.040000
24,001003,2004-06-30,0.040000,2.683000,0.040000
...,...,...,...,...,...
3280886,356289,2024-06-30,6.022442,27.191000,6.022442
3280887,356687,2023-05-31,1.010000,126.306000,1.010000
3280888,356687,2023-06-30,1.010000,126.306000,1.010000
3280899,356687,2024-05-31,0.540087,205.009227,0.540087


In [16]:
mktcap_comp_monthly= mktcap_comp_re_monthly.rename(columns={
    'prccd': 'prccm',
    'cshoc': 'cshom',
    'prccd_vw': 'prccm_vw'})
#mktcap_comp_monthly= mktcap_comp_re_monthly.drop('iid_count', axis=1)

In [17]:
mktcap_comp_monthly['tot_mktcap_comp'] = mktcap_comp_monthly['prccm'] * mktcap_comp_monthly['cshom']

In [23]:
file_path_2 = 'RR_data/mktcap_comp_monthly.csv' 
mktcap_comp_monthly.to_csv(file_path_2, index=False)

In [18]:
mktcap_comp_monthly

,gvkey,date,prccm,cshom,prccm_vw,tot_mktcap_comp
0,001003,2002-06-30,0.009000,2.683000,0.009000,0.024147
11,001003,2003-05-31,0.040000,2.683000,0.040000,0.107320
12,001003,2003-06-30,0.040000,2.683000,0.040000,0.107320
23,001003,2004-05-31,0.040000,2.683000,0.040000,0.107320
24,001003,2004-06-30,0.040000,2.683000,0.040000,0.107320
...,...,...,...,...,...,...
3283635,356289,2024-06-30,6.022442,27.191000,6.022442,163.756223
3283636,356687,2023-05-31,1.010000,126.306000,1.010000,127.569060
3283637,356687,2023-06-30,1.010000,126.306000,1.010000,127.569060
3283648,356687,2024-05-31,0.540087,205.009227,0.540087,110.722735


#### resampling testing

In [45]:
mktcap_comp_head = mktcap_comp.iloc[:10000]
mktcap_comp_mid = mktcap_comp.iloc[306668:316668]
mktcap_comp_end = mktcap_comp.iloc[-10000:]
mktcap_comp_test = pd.concat([mktcap_comp_head, mktcap_comp_mid, mktcap_comp_end]).reset_index(drop=True)

In [46]:
unique_date = mktcap_comp_test['date'].unique()
unique_date

<DatetimeArray>
['1999-05-03 00:00:00', '1999-06-10 00:00:00', '1999-06-11 00:00:00',
 '2012-06-29 00:00:00']
Length: 4, dtype: datetime64[ns]

In [29]:
'''mktcap_comp_test = mktcap_comp[1:100000]
mktcap_comp_test['date'].dtype'''

"mktcap_comp_test = mktcap_comp[1:100000]\nmktcap_comp_test['date'].dtype"

In [47]:
mktcap_comp_test_resample = mktcap_comp_test
mktcap_comp_test_resample['date'] = pd.to_datetime(mktcap_comp_test_resample['date'])
mktcap_comp_test_resample.set_index('date', inplace=True)
mktcap_comp_test_monthly = mktcap_comp_test_resample.groupby('gvkey').resample('ME', include_groups=False).mean().reset_index().dropna()

In [42]:
'''unique_date = mktcap_comp_test_monthly['date'].unique()
unique_date'''

<DatetimeArray>
['1999-05-31 00:00:00', '2012-06-30 00:00:00']
Length: 2, dtype: datetime64[ns]

In [48]:
mktcap_comp_test_monthly

,gvkey,date,prccd,cshoc,iid_count
0,001004,1999-05-31,19.1250,27443000.0,1.0
1,001004,1999-06-30,18.6875,27443000.0,1.0
2,001013,1999-05-31,46.4375,134808000.0,1.0
3,001013,1999-06-30,46.7500,134808000.0,1.0
4,001019,1999-05-31,228.0000,193000.0,1.0
...,...,...,...,...,...
695161,294524,2012-06-30,40.2700,574624000.0,1.0
695162,295113,2012-06-30,1.2800,558756000.0,1.0
695163,296318,2012-06-30,13.5100,134808000.0,1.0
695164,296753,2012-06-30,13.0100,54385000.0,1.0


In [105]:
print(mktcap_comp_test_monthly.loc[mktcap_comp_test_monthly['gvkey'] == '001003'])

     gvkey iid      cusip       date  mrkcap_comp  tot_mrkcap_comp
0   001003  01  000354100 2006-06-30     0.469525         0.469525
59  001003  01  000354100 2011-05-31     0.053660         0.053660
60  001003  01  000354100 2011-06-30     0.053660         0.053660
96  001003  01  000354100 2014-06-30     0.053660         0.053660


In [135]:
print(mktcap_comp.loc[mktcap_comp['gvkey'] == '160171'])

             date   gvkey  tot_mrkcap_comp
466913 2020-05-19  160171         0.225000
467400 2009-05-11  160171         0.500000
474190 2021-06-24  160171         0.004104
482614 2007-05-08  160171         4.000000
483601 2009-06-23  160171         0.750000
...           ...     ...              ...
270011 2020-06-25  160171         0.125000
271486 2005-06-01  160171              NaN
283755 2021-06-17  160171         0.004104
310284 2004-06-02  160171              NaN
316347 2006-05-04  160171              NaN

[348 rows x 3 columns]


## Quarterly Security Info - cshoq

### get all shares info(nontraded share classes) from quarterly financial report

In [192]:
cshoq_1 = conn.raw_sql("""
                        SELECT 
                        gvkey,
                        datadate,
                        cshoq,
                        fyr
                        
                        FROM comp.fundq
                
                        WHERE cshoq IS NOT NULL
                        """)

cshoq_1['datadate'] = pd.to_datetime(cshoq_1['datadate'])

In [193]:
cshoq_1

,gvkey,datadate,cshoq,fyr
0,001000,1966-03-31,0.219,12
1,001000,1966-06-30,0.219,12
2,001000,1966-09-30,0.219,12
3,001000,1966-12-31,0.219,12
4,001000,1967-03-31,0.277,12
...,...,...,...,...
267011,356128,2024-03-31,189.333,12
267012,356128,2024-06-30,190.016,12
267013,356289,2022-12-31,16.000,6
267014,356289,2023-03-31,26.800,6


In [194]:
gvkey_001003 = cshoq_1.loc[cshoq_1['gvkey'] == '001003']
print(gvkey_001003)

     gvkey   datadate  cshoq  fyr
60  001003 1982-12-31  2.100   12
61  001003 1983-09-30  2.100   12
62  001003 1983-12-31  2.683   12
63  001003 1984-03-31  2.683   12
64  001003 1984-06-30  2.683   12
65  001003 1984-09-30  2.683   12
66  001003 1984-12-31  2.683   12
67  001003 1986-01-31  2.683    1
68  001003 1986-04-30  2.683    1
69  001003 1986-07-31  2.683    1
70  001003 1986-10-31  2.683    1
71  001003 1987-01-31  2.683    1
72  001003 1987-04-30  2.683    1
73  001003 1987-07-31  2.683    1
74  001003 1987-10-31  2.683    1
75  001003 1988-01-31  2.683    1
76  001003 1988-04-30  2.683    1
77  001003 1988-07-31  2.683    1
78  001003 1988-10-31  2.683    1
79  001003 1989-01-31  2.683    1
80  001003 1989-07-31  2.683    1
81  001003 1990-01-31  2.683    1
82  001003 1990-04-30  2.683    1
83  001003 1990-07-31  2.683    1
84  001003 1990-10-31  2.033    1


In [24]:
cshoq_2 = conn.raw_sql("""
                        SELECT 
                        a.gvkey, a.datadate, a.cshoq, a.fyr, 
                        b.iid, b.ajexm AS ajexq
                        
                        FROM comp.fundq AS a
                        
                        LEFT JOIN comp.secm AS b
                        
                        ON a.gvkey = b.gvkey
                        AND DATE_TRUNC('month', a.datadate) + INTERVAL '1 MONTH - 1 day' = DATE_TRUNC('month', b.datadate) + INTERVAL '1 MONTH - 1 day'

                        WHERE a.cshoq IS NOT NULL
                        """)
#replace missing ajexq by ajexm from secm

In [25]:
cshoq_2['datadate'] = pd.to_datetime(cshoq_2['datadate'])
cshoq_2

,gvkey,datadate,cshoq,fyr,iid,ajexq
0,001000,1966-03-31,0.219,12,None,NaN
1,001000,1966-06-30,0.219,12,None,NaN
2,001000,1966-09-30,0.219,12,None,NaN
3,001000,1966-12-31,0.219,12,None,NaN
4,001000,1967-03-31,0.277,12,None,NaN
...,...,...,...,...,...,...
407080,356289,2023-03-31,26.800,6,None,NaN
407081,356289,2023-06-30,26.800,6,01,1.0
407082,356289,2023-09-30,26.857,6,01,1.0
407083,356289,2023-12-31,27.136,6,01,1.0


In [88]:
gvkey_001076_1 = cshoq_1.loc[cshoq_1['gvkey'] == '001076']
gv1076_sorted_1 = gvkey_001076_1.sort_values(by='datadate').reset_index().drop('index', axis=1)
print(gv1076_sorted_1.iloc[48:56, :])

     gvkey   datadate  cshoq  fyr
48  001076 1994-09-30  9.726    3
49  001076 1994-12-31  9.737    3
50  001076 1995-03-31  9.820   12
51  001076 1995-03-31  9.820    3
52  001076 1995-06-30  9.710   12
53  001076 1995-09-30  9.713   12
54  001076 1995-12-31  9.638   12
55  001076 1996-03-31  9.532   12


In [89]:
gvkey_001076_2 = cshoq_2.loc[cshoq_2['gvkey'] == '001076']
gv1076_sorted_2 = gvkey_001076_2.sort_values(by='datadate').reset_index().drop('index', axis=1)
print(gv1076_sorted_2.iloc[58:69, :])

     gvkey   datadate  cshoq  fyr iid  ajexq
58  001076 1994-12-31  9.737    3  02  6.750
59  001076 1994-12-31  9.737    3  01  3.375
60  001076 1995-03-31  9.820   12  01  3.375
61  001076 1995-03-31  9.820   12  02  6.750
62  001076 1995-03-31  9.820    3  02  6.750
63  001076 1995-03-31  9.820    3  01  3.375
64  001076 1995-06-30  9.710   12  02  6.750
65  001076 1995-06-30  9.710   12  01  3.375
66  001076 1995-09-30  9.713   12  01  3.375
67  001076 1995-09-30  9.713   12  02  6.750
68  001076 1995-12-31  9.638   12  02  6.750


In [172]:
gvkey_001076 = cshoq_2.loc[cshoq_2['gvkey'] == '001076']
gv1076_sorted = gvkey_001076.sort_values(by='datadate').reset_index().drop('index', axis=1)
print(gv1076_sorted.iloc[88:109, :])

      gvkey   datadate   cshoq  fyr iid  ajexq
88   001076 1998-06-30  21.091   12  02  3.375
89   001076 1998-06-30  21.091   12  01  3.375
90   001076 1998-09-30  20.909   12  01  3.375
91   001076 1998-09-30  20.909   12  02  3.375
92   001076 1998-12-31  20.549   12  01  3.375
93   001076 1998-12-31  20.549   12  02  3.375
94   001076 1999-03-31  20.039   12  01  3.375
95   001076 1999-03-31  20.039   12  02  3.375
96   001076 1999-06-30  20.085   12  01  3.375
97   001076 1999-06-30  20.085   12  02  3.375
98   001076 1999-09-30  19.916   12  01  3.375
99   001076 1999-09-30  19.916   12  02  3.375
100  001076 1999-12-31  19.923   12  01  3.375
101  001076 1999-12-31  19.923   12  02  3.375
102  001076 2000-03-31  19.894   12  02  3.375
103  001076 2000-03-31  19.894   12  01  3.375
104  001076 2000-06-30  19.759   12  01  3.375
105  001076 2000-06-30  19.759   12  02  3.375
106  001076 2000-09-30  19.855   12  01  3.375
107  001076 2000-09-30  19.855   12  02  3.375
108  001076 2

In [165]:
gvkey_356128 = cshoq_2.loc[cshoq_2['gvkey'] == '356128']
gv1076_sorted_356128 = gvkey_356128.sort_values(by='datadate').reset_index().drop('index', axis=1)
print(gv1076_sorted_356128)

    gvkey   datadate    cshoq  fyr iid  ajexq
0  356128 2022-12-31  190.310   12  90    1.0
1  356128 2023-09-30  189.685   12  90    1.0
2  356128 2023-12-31  189.333   12  90    1.0
3  356128 2024-03-31  189.333   12  90    1.0
4  356128 2024-06-30  190.016   12  90    1.0


In [92]:
'''
1. DATE_TRUNC('month', date) + INTERVAL '1 MONTH - 1 day' make the date as the end of each month, important for merge and data population
2. also check cshoq 1&2 to see diifference and merge result
'''

"\n1. DATE_TRUNC('month', date) + INTERVAL '1 MONTH - 1 day' make the date as the end of each month, important for merge and data population\n2. also check cshoq 1&2 to see diifference and merge result\n"

In [26]:
#select distinct
cshoq_2_drop = cshoq_2.drop('fyr', axis=1)
cshoq_m = cshoq_2_drop.drop_duplicates(subset=['gvkey','iid' , 'datadate', 'cshoq', 'ajexq'])
cshoq_m

,gvkey,datadate,cshoq,iid,ajexq
0,001000,1966-03-31,0.219,None,NaN
1,001000,1966-06-30,0.219,None,NaN
2,001000,1966-09-30,0.219,None,NaN
3,001000,1966-12-31,0.219,None,NaN
4,001000,1967-03-31,0.277,None,NaN
...,...,...,...,...,...
407080,356289,2023-03-31,26.800,None,NaN
407081,356289,2023-06-30,26.800,01,1.0
407082,356289,2023-09-30,26.857,01,1.0
407083,356289,2023-12-31,27.136,01,1.0


In [27]:
cshoq_sort = cshoq_m.sort_values(by=['gvkey', 'iid', 'datadate']).reset_index().drop('index', axis=1)
cshoq_group = cshoq_sort.groupby(['gvkey', 'iid', 'datadate'])
idx_max = cshoq_group['cshoq'].idxmax()
cshoq_max = cshoq_sort.loc[idx_max].reset_index(drop=True)
cshoq_max

,gvkey,datadate,cshoq,iid,ajexq
0,001000,1970-09-30,2.230,01,1.0
1,001000,1970-12-31,2.446,01,1.0
2,001000,1971-03-31,3.507,01,1.0
3,001000,1971-06-30,3.518,01,1.0
4,001000,1971-09-30,3.515,01,1.0
...,...,...,...,...,...
1767691,356128,2024-06-30,190.016,90,1.0
1767692,356289,2023-06-30,26.800,01,1.0
1767693,356289,2023-09-30,26.857,01,1.0
1767694,356289,2023-12-31,27.136,01,1.0


In [28]:
cshoq_max= cshoq_max.rename(columns={
    'datadate': 'date'})

In [29]:
cshoq_merge = cshoq_max.groupby(['gvkey', 'date']).agg({
    'cshoq': 'mean',  
    'ajexq': 'mean',   
}).reset_index()
cshoq_merge

,gvkey,date,cshoq,ajexq
0,001000,1970-09-30,2.230,1.0
1,001000,1970-12-31,2.446,1.0
2,001000,1971-03-31,3.507,1.0
3,001000,1971-06-30,3.518,1.0
4,001000,1971-09-30,3.515,1.0
...,...,...,...,...
1628002,356128,2024-06-30,190.016,1.0
1628003,356289,2023-06-30,26.800,1.0
1628004,356289,2023-09-30,26.857,1.0
1628005,356289,2023-12-31,27.136,1.0


In [32]:
'''
#keep max cshoq
cshoq_max = cshoq_m.loc[cshoq_m.groupby(['gvkey', 'iid', 'datadate'])['cshoq'].idxmax()]
cshoq_max
#return more rows than expected - check for duplicates
duplicate_rows = cshoq_m.duplicated(subset=['gvkey', 'iid', 'datadate'], keep=False)
print(cshoq_m[duplicate_rows])
'''

"\n#keep max cshoq\ncshoq_max = cshoq_m.loc[cshoq_m.groupby(['gvkey', 'iid', 'datadate'])['cshoq'].idxmax()]\ncshoq_max\n#return more rows than expected - check for duplicates\nduplicate_rows = cshoq_m.duplicated(subset=['gvkey', 'iid', 'datadate'], keep=False)\nprint(cshoq_m[duplicate_rows])\n"

In [138]:
# TESTING
cshoq_max_test = cshoq_max[0:10000]
cshoq_max_test.set_index('datadate', inplace=True)

# Perform forward fill within each group
cshoq_max_test_fill = cshoq_max_test.groupby(['gvkey', 'iid']).resample('ME', include_groups=False).ffill().reset_index()

print(cshoq_max_test_fill)

        gvkey iid   datadate  cshoq  fyr  ajexq
0      001000  01 1970-09-30  2.230   12    1.0
1      001000  01 1970-10-31  2.230   12    1.0
2      001000  01 1970-11-30  2.230   12    1.0
3      001000  01 1970-12-31  2.446   12    1.0
4      001000  01 1971-01-31  2.446   12    1.0
...       ...  ..        ...    ...  ...    ...
32396  001210  03 2021-08-31  2.882    3    NaN
32397  001210  03 2021-09-30  2.882    3    NaN
32398  001210  03 2021-10-31  2.882    3    NaN
32399  001210  03 2021-11-30  2.882    3    NaN
32400  001210  03 2021-12-31  2.882    3    NaN

[32401 rows x 6 columns]


In [30]:
cshoq_merge.set_index('date', inplace=True)

# Perform forward fill within each group/Upsample to monthly frequency
cshoq_ffill = cshoq_merge.groupby('gvkey').resample('ME', include_groups=False).ffill().reset_index()

print(cshoq_ffill)

          gvkey       date   cshoq  ajexq
0        001000 1970-09-30   2.230    1.0
1        001000 1970-10-31   2.230    1.0
2        001000 1970-11-30   2.230    1.0
3        001000 1970-12-31   2.446    1.0
4        001000 1971-01-31   2.446    1.0
...         ...        ...     ...    ...
5095855  356289 2023-11-30  26.857    1.0
5095856  356289 2023-12-31  27.136    1.0
5095857  356289 2024-01-31  27.136    1.0
5095858  356289 2024-02-29  27.136    1.0
5095859  356289 2024-03-31  27.136    1.0

[5095860 rows x 4 columns]


In [32]:
cshoq_ffill['date'] = pd.to_datetime(cshoq_ffill['date'])
cshoq_ffill['year'] = cshoq_ffill['date'].dt.year
cshoq_ffill['month_day'] = cshoq_ffill['date'].dt.strftime('%m-%d')

russell_month_day = '05-31'
cshoq_ffill_may = cshoq_ffill[cshoq_ffill['month_day'] == russell_month_day]

In [35]:
file_path_3 = 'RR_data/cshoq_ffill_may.csv' 
cshoq_ffill_may.to_csv(file_path_3, index=False)

In [185]:
#keep only the May data of each year and test
print(cshoq_ffill_may.loc[cshoq_ffill_may['gvkey'] == '001000'])

     gvkey       date  cshoq  ajexq  year month_day
8   001000 1971-05-31  3.507    1.0  1971     05-31
20  001000 1972-05-31  2.981    1.0  1972     05-31
32  001000 1973-05-31  3.050    1.0  1973     05-31
44  001000 1974-05-31  2.610    1.0  1974     05-31
56  001000 1975-05-31  2.092    1.0  1975     05-31
68  001000 1976-05-31  2.096    1.0  1976     05-31
80  001000 1977-05-31  2.209    1.0  1977     05-31


In [188]:
print(cshoq_ffill_may.loc[cshoq_ffill_may['gvkey'] == '001003'])

      gvkey       date  cshoq  ajexq  year month_day
127  001003 1984-05-31  2.683    1.0  1984     05-31
139  001003 1985-05-31  2.683    1.0  1985     05-31
151  001003 1986-05-31  2.683    1.0  1986     05-31
163  001003 1987-05-31  2.683    1.0  1987     05-31
175  001003 1988-05-31  2.683    1.0  1988     05-31
187  001003 1989-05-31  2.683    1.0  1989     05-31
199  001003 1990-05-31  2.683    1.0  1990     05-31


In [34]:
#keep only the May data of each year and test
print(cshoq_ffill_may.loc[cshoq_ffill_may['gvkey'] == '001000'])

     gvkey       date  cshoq  ajexq  year month_day
8   001000 1971-05-31  3.507    1.0  1971     05-31
20  001000 1972-05-31  2.981    1.0  1972     05-31
32  001000 1973-05-31  3.050    1.0  1973     05-31
44  001000 1974-05-31  2.610    1.0  1974     05-31
56  001000 1975-05-31  2.092    1.0  1975     05-31
68  001000 1976-05-31  2.096    1.0  1976     05-31
80  001000 1977-05-31  2.209    1.0  1977     05-31


In [33]:
print(cshoq_ffill.loc[cshoq_ffill['gvkey'] == '001000'])

     gvkey       date  cshoq  ajexq  year month_day
0   001000 1970-09-30  2.230    1.0  1970     09-30
1   001000 1970-10-31  2.230    1.0  1970     10-31
2   001000 1970-11-30  2.230    1.0  1970     11-30
3   001000 1970-12-31  2.446    1.0  1970     12-31
4   001000 1971-01-31  2.446    1.0  1971     01-31
..     ...        ...    ...    ...   ...       ...
86  001000 1977-11-30  2.226    1.0  1977     11-30
87  001000 1977-12-31  2.226    1.0  1977     12-31
88  001000 1978-01-31  2.226    1.0  1978     01-31
89  001000 1978-02-28  2.226    1.0  1978     02-28
90  001000 1978-03-31  2.227    1.0  1978     03-31

[91 rows x 6 columns]


## Monthly Security Info

### add ajexm and cusip(first 6 digits for company id) info to mktcap_comp

In [7]:
secm_1 = conn.raw_sql("""
                        SELECT DISTINCT
                        gvkey,
                        LEFT(cusip, 6) AS cusip_co,
                        datadate AS date, 
                        ajexm
                        
                        FROM comp.secm
                        
                        WHERE EXTRACT(MONTH FROM datadate) IN (5,6)
                        AND datadate >= '1999-01-01' 
                        AND curcdm = 'USD'
                        AND tpci IN ('0')
                        
                        """)

In [37]:
secm_1['date'] = pd.to_datetime(secm_1['date'])
secm_1

,gvkey,cusip_co,date,ajexm
0,001003,000354,2002-06-30,1.0
1,001003,000354,2003-05-31,1.0
2,001003,000354,2003-06-30,1.0
3,001003,000354,2004-05-31,1.0
4,001003,000354,2004-06-30,1.0
...,...,...,...,...
59554,356289,83417Y,2024-06-30,1.0
59555,356687,046824,2023-05-31,1.0
59556,356687,046824,2023-06-30,1.0
59557,356687,046824,2024-05-31,1.0


In [8]:
print(secm_1.loc[secm_1['gvkey'] == '001076'])

      gvkey cusip_co        date  ajexm
486  001076   002535  1999-05-31  3.375
487  001076   002535  1999-06-30  3.375
488  001076   002535  2000-05-31  3.375
489  001076   002535  2000-06-30  3.375
490  001076   002535  2001-05-31  3.375
..      ...      ...         ...    ...
557  001076   74319R  2022-06-30  1.000
558  001076   74319R  2023-05-31  1.000
559  001076   74319R  2023-06-30  1.000
560  001076   74319R  2024-05-31  1.000
561  001076   74319R  2024-06-30  1.000

[76 rows x 4 columns]


In [55]:
print(secm_1.loc[secm_1['gvkey'] == '001204'])

Empty DataFrame
Columns: [gvkey, cusip_co, date, ajexm]
Index: []


In [98]:
len_column1 = len(secm_1['date'])
print(f"Length of date in secm: {len_column1}")

len_column2 = len(mktcap_comp_monthly['date'])
print(f"Length of date in mktcap_comp_monthly: {len_column2}")

Length of date in secm: 564003
Length of date in secm: 548653


In [112]:
'''secm_1['date'].dtype
mktcap_comp_monthly['date'].dtype
'''

"secm_1['date'].dtype\nmktcap_comp_monthly['date'].dtype\n"

In [38]:
mktcap_comp_mm = pd.merge(mktcap_comp_monthly, secm_1, on=['gvkey', 'date'], how='left')

In [39]:
mktcap_comp_mm

,gvkey,date,prccm,cshom,prccm_vw,tot_mktcap_comp,cusip_co,ajexm
0,001003,2002-06-30,0.009000,2.683000,0.009000,0.024147,000354,1.0
1,001003,2003-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0
2,001003,2003-06-30,0.040000,2.683000,0.040000,0.107320,000354,1.0
3,001003,2004-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0
4,001003,2004-06-30,0.040000,2.683000,0.040000,0.107320,000354,1.0
...,...,...,...,...,...,...,...,...
549709,356289,2024-06-30,6.022442,27.191000,6.022442,163.756223,83417Y,1.0
549710,356687,2023-05-31,1.010000,126.306000,1.010000,127.569060,046824,1.0
549711,356687,2023-06-30,1.010000,126.306000,1.010000,127.569060,046824,1.0
549712,356687,2024-05-31,0.540087,205.009227,0.540087,110.722735,046824,1.0


### Dealing with Nontraded Shares

In [40]:
mktcap_comp_mm['month'] = mktcap_comp_mm['date'].dt.month

# Filter to get only May data (month == 5)
mktcap_comp_may = mktcap_comp_mm[mktcap_comp_mm['month'] == 5]

# Optionally, drop the 'month' column if you don't need it
mktcap_comp_may = mktcap_comp_may.drop(columns=['month'])
mktcap_comp_may

,gvkey,date,prccm,cshom,prccm_vw,tot_mktcap_comp,cusip_co,ajexm
1,001003,2003-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0
3,001003,2004-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0
5,001003,2005-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0
7,001003,2006-05-31,0.190227,2.683000,0.190227,0.510380,000354,1.0
9,001003,2007-05-31,0.075000,2.683000,0.075000,0.201225,000354,1.0
...,...,...,...,...,...,...,...,...
549704,353742,2024-05-31,2.650506,38.317000,2.650506,101.559442,53000A,1.0
549706,356289,2023-05-31,4.702941,26.800000,4.702941,126.038824,83417Y,1.0
549708,356289,2024-05-31,5.976945,27.166000,5.976945,162.369700,83417Y,1.0
549710,356687,2023-05-31,1.010000,126.306000,1.010000,127.569060,046824,1.0


In [43]:
#find nontraded shares
mktcap_cshoq = pd.merge(mktcap_comp_may, cshoq_ffill_may, on=['gvkey', 'date'], how='left')

In [46]:
file_path_4 = 'RR_data/mktcap_cshoq.csv' 
mktcap_cshoq.to_csv(file_path_4, index=False)

In [44]:
mktcap_cshoq

,gvkey,date,prccm,cshom,prccm_vw,tot_mktcap_comp,cusip_co,ajexm,cshoq,ajexq,year,month_day
0,001003,2003-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0,NaN,NaN,NaN,NaN
1,001003,2004-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0,NaN,NaN,NaN,NaN
2,001003,2005-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0,NaN,NaN,NaN,NaN
3,001003,2006-05-31,0.190227,2.683000,0.190227,0.510380,000354,1.0,NaN,NaN,NaN,NaN
4,001003,2007-05-31,0.075000,2.683000,0.075000,0.201225,000354,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
275390,353742,2024-05-31,2.650506,38.317000,2.650506,101.559442,53000A,1.0,42.733,1.0,2024.0,05-31
275391,356289,2023-05-31,4.702941,26.800000,4.702941,126.038824,83417Y,1.0,NaN,NaN,NaN,NaN
275392,356289,2024-05-31,5.976945,27.166000,5.976945,162.369700,83417Y,1.0,NaN,NaN,NaN,NaN
275393,356687,2023-05-31,1.010000,126.306000,1.010000,127.569060,046824,1.0,NaN,NaN,NaN,NaN


In [47]:
print(mktcap_cshoq.loc[mktcap_cshoq['gvkey'] == '001004'])

     gvkey       date      prccm   cshom   prccm_vw  tot_mktcap_comp cusip_co  \
13  001004 1999-05-31  20.553125  27.443  20.553125       564.039409   000361   
14  001004 2000-05-31  14.715909  26.963  14.715909       396.785057   000361   
15  001004 2001-05-31  12.992727  26.932  12.992727       349.920131   000361   
16  001004 2002-05-31  11.997273  26.860  11.997273       322.246745   000361   
17  001004 2003-05-31   4.228095  31.850   4.228095       134.664833   000361   
18  001004 2004-05-31   9.287000  32.245   9.287000       299.459315   000361   
19  001004 2005-05-31  14.401905  32.438  14.401905       467.168987   000361   
20  001004 2006-05-31  25.520455  36.461  25.520455       930.501293   000361   
21  001004 2007-05-31  32.721818  37.294  32.721818      1220.327487   000361   
22  001004 2008-05-31  21.980000  38.738  21.980000       851.461240   000361   
23  001004 2009-05-31  14.808000  38.697  14.808000       573.025176   000361   
24  001004 2010-05-31  22.47

In [ ]:
# fill NAs for ajexm and ajexq
mktcap_cshoq['ajexm'] = mktcap_cshoq['ajexm'].fillna(1)
mktcap_cshoq['ajexq'] = mktcap_cshoq['ajexq'].fillna(mktcap_cshoq['ajexm'])

In [49]:
count_df = mktcap_cshoq.groupby(['gvkey', 'date']).size().reset_index(name='count')

# Check for uniqueness
count_df['is_unique'] = count_df['count'] == 1

# Find pairs that are not unique
not_unique_pairs = count_df[count_df['is_unique'] == False]

# Display results
print("Pairs that are not unique:")
print(not_unique_pairs)

Pairs that are not unique:
         gvkey       date  count  is_unique
244     001076 1999-05-31      2      False
245     001076 2000-05-31      2      False
246     001076 2001-05-31      2      False
247     001076 2002-05-31      2      False
248     001076 2003-05-31      2      False
...        ...        ...    ...        ...
258708  183565 2015-05-31      2      False
264807  186169 2016-05-31      2      False
269546  194755 2017-05-31      2      False
274245  296944 2014-05-31      2      False
274246  296944 2015-05-31      2      False

[336 rows x 4 columns]


In [51]:
print(mktcap_cshoq.loc[mktcap_cshoq['gvkey'] == '001204'])

Empty DataFrame
Columns: [gvkey, date, prccm, cshom, prccm_vw, tot_mktcap_comp, cusip_co, ajexm, cshoq, ajexq, year, month_day]
Index: []


In [50]:
print(mktcap_cshoq.loc[mktcap_cshoq['gvkey'] == '001076'])

      gvkey       date      prccm      cshom   prccm_vw  tot_mktcap_comp  \
244  001076 1999-05-31  16.915095  20.003900  16.964880       338.367861   
245  001076 1999-05-31  16.915095  20.003900  16.964880       338.367861   
246  001076 2000-05-31  14.161216  19.816545  14.199810       280.626383   
247  001076 2000-05-31  14.161216  19.816545  14.199810       280.626383   
248  001076 2001-05-31  17.067514  19.895818  17.087693       339.572150   
249  001076 2001-05-31  17.067514  19.895818  17.087693       339.572150   
250  001076 2002-05-31  27.073811  19.943364  27.078578       539.942848   
251  001076 2002-05-31  27.073811  19.943364  27.078578       539.942848   
252  001076 2003-05-31  22.519237  21.716571  22.534759       489.040608   
253  001076 2003-05-31  22.519237  21.716571  22.534759       489.040608   
254  001076 2004-05-31  27.851621  33.079650  27.896529       921.321872   
255  001076 2004-05-31  27.851621  33.079650  27.896529       921.321872   
256  001076 

### load data and continue calculation

In [25]:
mktcap_cshoq = pd.read_csv("RR_data/mktcap_cshoq.csv")
mktcap_comp_monthly = pd.read_csv("RR_data/mktcap_comp_monthly.csv")
mktcap_crsp_monthly = pd.read_csv("RR_data/mktcap_crsp_monthly.csv")

In [26]:
columns_to_drop = ['year', 'month_day']
mktcap_cshoq_all = mktcap_cshoq.drop(columns=columns_to_drop)

In [27]:
# fill NAs for ajexm and ajexq
mktcap_cshoq_all['ajexm'] = mktcap_cshoq_all['ajexm'].fillna(1)
mktcap_cshoq_all['ajexq'] = mktcap_cshoq_all['ajexq'].fillna(mktcap_cshoq['ajexm'])

In [28]:
mktcap_cshoq_all['cshoq'] = mktcap_cshoq_all['cshoq'].fillna(0)

In [29]:
# Calculating tot_mktcap_COMP_ALL
mktcap_cshoq_all['tot_mktcap_COMP_ALL'] = ((mktcap_cshoq_all['cshoq'] * mktcap_cshoq_all['ajexq'] - mktcap_cshoq_all['cshom'] * mktcap_cshoq_all['ajexm']) * mktcap_cshoq_all['prccm_vw'] / mktcap_cshoq_all['ajexm'] 
                             + mktcap_cshoq_all['tot_mktcap_comp'])

# If cshoq * ajexq - tot_cshom * ajexm < 0, set tot_mktcap_COMP_ALL to tot_mktcap_COMP
condition = (mktcap_cshoq_all['cshoq'] * mktcap_cshoq_all['ajexq'] - mktcap_cshoq_all['cshom'] * mktcap_cshoq_all['ajexm']) < 0
mktcap_cshoq_all.loc[condition, 'tot_mktcap_COMP_ALL'] = mktcap_cshoq_all['tot_mktcap_comp']

In [31]:
mktcap_cshoq_all

,gvkey,date,prccm,cshom,prccm_vw,tot_mktcap_comp,cusip_co,ajexm,cshoq,ajexq,tot_mktcap_COMP_ALL
0,1003,2003-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0,0.000,1.0,0.107320
1,1003,2004-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0,0.000,1.0,0.107320
2,1003,2005-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0,0.000,1.0,0.107320
3,1003,2006-05-31,0.190227,2.683000,0.190227,0.510380,000354,1.0,0.000,1.0,0.510380
4,1003,2007-05-31,0.075000,2.683000,0.075000,0.201225,000354,1.0,0.000,1.0,0.201225
...,...,...,...,...,...,...,...,...,...,...,...
275390,353742,2024-05-31,2.650506,38.317000,2.650506,101.559442,53000A,1.0,42.733,1.0,113.264077
275391,356289,2023-05-31,4.702941,26.800000,4.702941,126.038824,83417Y,1.0,0.000,1.0,126.038824
275392,356289,2024-05-31,5.976945,27.166000,5.976945,162.369700,83417Y,1.0,0.000,1.0,162.369700
275393,356687,2023-05-31,1.010000,126.306000,1.010000,127.569060,046824,1.0,0.000,1.0,127.569060


In [34]:
mktcap_crsp_monthly

,permno,permco,cusip,date,mrkcap_crsp,tot_mrkcap_crsp
0,10001,7953,36720410,2000-05-31,19.686333,19.686333
1,10001,7953,36720410,2000-06-30,19.250500,19.250500
2,10001,7953,36720410,2000-07-31,NaN,NaN
3,10001,7953,36720410,2000-08-31,NaN,NaN
4,10001,7953,36720410,2000-09-30,NaN,NaN
...,...,...,...,...,...,...
675547,93105,7427,92925187,2008-02-29,NaN,NaN
675548,93105,7427,92925187,2008-03-31,NaN,NaN
675549,93105,7427,92925187,2008-04-30,NaN,NaN
675550,93105,7427,92925187,2008-05-31,172.952381,172.952381


In [33]:
file_path_5 = 'RR_data/mktcap_comp_all.csv' 
mktcap_cshoq_all.to_csv(file_path_5, index=False)

## Merging and Caculating Ranks

In [39]:
# If 'id' is not already a string, convert it first and then take the first 6 digits
mktcap_crsp_monthly['cusip_co'] = mktcap_crsp_monthly['cusip'].astype(str).str[:6]

In [36]:
mktcap_cshoq_all

,gvkey,date,prccm,cshom,prccm_vw,tot_mktcap_comp,cusip_co,ajexm,cshoq,ajexq,tot_mktcap_COMP_ALL
0,1003,2003-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0,0.000,1.0,0.107320
1,1003,2004-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0,0.000,1.0,0.107320
2,1003,2005-05-31,0.040000,2.683000,0.040000,0.107320,000354,1.0,0.000,1.0,0.107320
3,1003,2006-05-31,0.190227,2.683000,0.190227,0.510380,000354,1.0,0.000,1.0,0.510380
4,1003,2007-05-31,0.075000,2.683000,0.075000,0.201225,000354,1.0,0.000,1.0,0.201225
...,...,...,...,...,...,...,...,...,...,...,...
275390,353742,2024-05-31,2.650506,38.317000,2.650506,101.559442,53000A,1.0,42.733,1.0,113.264077
275391,356289,2023-05-31,4.702941,26.800000,4.702941,126.038824,83417Y,1.0,0.000,1.0,126.038824
275392,356289,2024-05-31,5.976945,27.166000,5.976945,162.369700,83417Y,1.0,0.000,1.0,162.369700
275393,356687,2023-05-31,1.010000,126.306000,1.010000,127.569060,046824,1.0,0.000,1.0,127.569060


In [40]:
#find nontraded shares
Russell_1 = pd.merge(mktcap_crsp_monthly, mktcap_cshoq_all, on=['cusip_co', 'date'], how='left')

In [41]:
Russell_1

,permno,permco,cusip,date,mrkcap_crsp,tot_mrkcap_crsp,cusip_no,cusip_co,gvkey,prccm,cshom,prccm_vw,tot_mktcap_comp,ajexm,cshoq,ajexq,tot_mktcap_COMP_ALL
0,10001,7953,36720410,2000-05-31,19.686333,19.686333,367204,367204,12994.0,8.053977,2.457,8.053977,19.788622,1.50,2.464,1.50,19.845000
1,10001,7953,36720410,2000-06-30,19.250500,19.250500,367204,367204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,7953,36720410,2000-07-31,NaN,NaN,367204,367204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,7953,36720410,2000-08-31,NaN,NaN,367204,367204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,7953,36720410,2000-09-30,NaN,NaN,367204,367204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675763,93105,7427,92925187,2008-02-29,NaN,NaN,929251,929251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675764,93105,7427,92925187,2008-03-31,NaN,NaN,929251,929251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675765,93105,7427,92925187,2008-04-30,NaN,NaN,929251,929251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675766,93105,7427,92925187,2008-05-31,172.952381,172.952381,929251,929251,17157.0,1.080952,160.000,1.080952,172.952381,0.02,164.907,0.02,178.256614


In [46]:
# Assuming 'Russell_4' is the initial DataFrame
Russell_2 = Russell_1.copy()

# 1. Use CRSP market cap
Russell_2['tot_mktcap_r3'] = Russell_2['tot_mrkcap_crsp']

# 2. If missing, use Compustat market cap
Russell_2['tot_mktcap_r3'] = Russell_2['tot_mktcap_r3'].fillna(Russell_2['tot_mktcap_comp'])

# 3. Use Compustat total market cap (tot_mktcap_COMP_ALL) if it's higher due to OTC shares or non-tradable shares
Russell_2['tot_mktcap_r3'] = Russell_2.apply(
    lambda row: row['tot_mktcap_COMP_ALL'] if row['tot_mktcap_COMP_ALL'] > row['tot_mktcap_r3'] else row['tot_mktcap_r3'],
    axis=1
)

In [48]:
Russell_2['mkt_value'] = Russell_2['tot_mktcap_r3']
Russell_2['mkt_value'] = pd.to_numeric(Russell_2['mkt_value'], errors='coerce')

In [44]:
#present in dollar format
#Russell_2['mkt_value'] = Russell_2['tot_mktcap_r3'].apply(lambda x: f"${x:,.3f}")

In [50]:
# Step 1: Filter rows where 'mkt_value' > 0
Russell_2_filtered = Russell_2[Russell_2['mkt_value'] > 0]

# Step 2: Rank the 'tot_mktcap_r3' within each 'date' group, in descending order
Russell_2_filtered['Rank'] = Russell_2_filtered.groupby('date')['tot_mktcap_r3'].rank(ascending=False)

# Step 3: Output the ranked DataFrame into 'Russell_6'
Russell_3 = Russell_2_filtered.copy()

# Optional: Reset index if needed
Russell_3.reset_index(drop=True, inplace=True)

In [51]:
Russell_3

,permno,permco,cusip,date,mrkcap_crsp,tot_mrkcap_crsp,cusip_no,cusip_co,gvkey,prccm,cshom,prccm_vw,tot_mktcap_comp,ajexm,cshoq,ajexq,tot_mktcap_COMP_ALL,tot_mktcap_r3,mkt_value,Rank
0,10001,7953,36720410,2000-05-31,19.686333,19.686333,367204,367204,12994.0,8.053977,2.457000,8.053977,19.788622,1.50,2.464,1.50,19.845000,19.845000,19.845000,7301.0
1,10001,7953,36720410,2000-06-30,19.250500,19.250500,367204,367204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.250500,19.250500,7291.0
2,10001,7953,36720410,2001-05-31,25.670491,25.670491,367204,367204,12994.0,10.231364,2.491000,10.231364,25.486327,1.50,2.509,1.50,25.670491,25.670491,25.670491,6363.0
3,10001,7953,36720410,2001-06-30,28.589913,28.589913,367204,367204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.589913,28.589913,6121.0
4,10001,7953,36720410,2002-05-31,25.102251,25.102251,367204,367204,12994.0,9.763614,2.569818,9.763614,25.090712,1.50,2.571,1.50,25.102251,25.102251,25.102251,6031.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130635,93105,7427,92925187,2006-06-30,1104.785711,1104.785711,929251,929251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1104.785711,1104.785711,2031.0
130636,93105,7427,92925187,2007-05-31,861.667004,861.667004,929251,929251,17157.0,5.225455,164.898000,5.225455,861.667004,0.02,164.898,0.02,861.667004,861.667004,861.667004,2730.0
130637,93105,7427,92925187,2007-06-30,788.526531,788.526531,929251,929251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,788.526531,788.526531,2772.0
130638,93105,7427,92925187,2008-05-31,172.952381,172.952381,929251,929251,17157.0,1.080952,160.000000,1.080952,172.952381,0.02,164.907,0.02,178.256614,178.256614,178.256614,4574.0


In [52]:
file_path_6 = 'RR_data/Russell_3_rank.csv' 
Russell_3.to_csv(file_path_5, index=False)